In [ ]:
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
analyze_path = os.path.join(parent_dir, "utils")

os.chdir(analyze_path)

要先建立輸入到模型的資料
- 若是要分類是否是熱點，應該要以一個區域的grid為單位
- 所以建立得grid亦包含該地區的所有特徵資料，以比例顯示

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

from utils import get_grid, calculate_gi

from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, average_precision_score

In [ ]:
dataA1 = pd.read_csv('../ComputedData/Accident/DataA1_with_MYP.csv')
dataA2 = pd.read_csv('../ComputedData/Accident/DataA2_with_MYP.csv')
taiwan = gpd.read_file('../Data/OFiles_9e222fea-bafb-4436-9b17-10921abc6ef2/TOWN_MOI_1140318.shp')
taiwan = taiwan[(~taiwan['TOWNNAME'].isin(['旗津區', '頭城鎮', '蘭嶼鄉', '綠島鄉', '琉球鄉'])) & 
                (~taiwan['COUNTYNAME'].isin(['金門縣', '連江縣', '澎湖縣']))]

filtered_A2 = dataA2[dataA2['當事者順位'] == 1]
filtered_A1 = dataA1[dataA1['當事者順位'] == 1]

filtered_A1['source'] = 'A1'
filtered_A2['source'] = 'A2'
filtered_A1['num_accidents'] = 1 
filtered_A2['num_accidents'] = 1
combined_data = pd.concat([filtered_A1, filtered_A2], ignore_index=True)

hex_grid = get_grid(combined_data, hex_size=0.01, threshold=-1)
taiwan = taiwan.to_crs(hex_grid.crs)

hex_grid = hex_grid[hex_grid.intersects(taiwan.unary_union)]
# hex_grid.to_csv('../ComputedData/Grid/hex_grid.csv', index=False)

In [ ]:
# grid_gi = calculate_gi(6, hex_grid, adjacency='knn')
# grid_gi.to_csv('../ComputedData/Grid/grid_gi.csv', index=False)

hex_grid目前包含所有事故索引，所以要回推
- 回推方式從combined_data獲取，並且計算他們的事故特徵平均

In [ ]:
hex_grid = pd.read_csv('../ComputedData/Grid/hex_grid.csv')

# 讀取後會變字串
from shapely import wkt
import geopandas as gpd

# 先轉成 shapely 物件
hex_grid['geometry'] = hex_grid['geometry'].apply(wkt.loads)
# 再轉回 GeoDataFrame
hex_grid = gpd.GeoDataFrame(hex_grid, geometry='geometry', crs=taiwan.crs)  # 記得填入原本的 CRS
# 計算 centroid
centroids = hex_grid.centroid

In [ ]:
import ast

# change it to list
grid_gi = pd.read_csv('../ComputedData/Grid/grid_gi.csv')
grid_gi['accident_indices'] = grid_gi['accident_indices'].apply(ast.literal_eval)
grid_gi['accident_indices'].iloc[0]

## obtain hotspot's county

In [ ]:
# dissolve to county
taiwan_cnty = taiwan[['COUNTYNAME', 'geometry']].dissolve(by='COUNTYNAME').reset_index()

# centroid
hex_cent = hex_grid.copy()
hex_cent['geometry'] = hex_cent.geometry.centroid
county_join = gpd.sjoin(hex_cent, taiwan_cnty, how='left', predicate='within')[['COUNTYNAME']]

# 把縣市結果貼回原 hex_grid（筆數不變）
grid = hex_grid.join(county_join)
# 如需把縣市帶回 GI 結果（假設索引一致）
grid = grid_gi.reset_index(inplace=False).join(grid[['COUNTYNAME']])

In [ ]:
select_group = [
    # 氣候暫不討論
    # '天候名稱', '光線名稱',

    # 道路問題
    '路面狀況-路面鋪裝名稱', '路面狀況-路面狀態名稱', '路面狀況-路面缺陷名稱',
    '道路障礙-障礙物名稱', '道路障礙-視距品質名稱', '道路障礙-視距名稱',

    # 號誌
    '號誌-號誌種類名稱', '號誌-號誌動作名稱',

    # 車道劃分
    '車道劃分設施-分道設施-快車道或一般車道間名稱',
    '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',

    # 大類別
    # '肇因研判大類別名稱-主要', '肇因研判大類別名稱-個別', # 聚焦道路類型
    # '當事者區分-類別-大類別名稱-車種', # 聚焦道路類型
    # '當事者行動狀態大類別名稱', # 聚焦道路類型
    # '車輛撞擊部位大類別名稱-最初', '車輛撞擊部位大類別名稱-其他', # 道路類型很大程度影響撞擊部位，所以不考慮
    '事故類型及型態大類別名稱', '車道劃分設施-分向設施大類別名稱',
    '事故位置大類別名稱', '道路型態大類別名稱',
    
    # 子類別
    # '肇因研判子類別名稱-主要', '肇因研判子類別名稱-個別', # 聚焦道路類型
    # '當事者區分-類別-子類別名稱-車種', # 聚焦道路類型
    # '當事者行動狀態子類別名稱', # 聚焦道路類型
    # '車輛撞擊部位子類別名稱-最初', '車輛撞擊部位子類別名稱-其他', # 道路類型很大程度影響撞擊部位，所以不考慮
    # '事故類型及型態子類別名稱', '車道劃分設施-分向設施子類別名稱', 
    # '事故位置子類別名稱', '道路型態子類別名稱',

    # 其他
    # '當事者屬-性-別名稱', '當事者事故發生時年齡', '速限-第1當事者', '道路類別-第1當事者-名稱',
    # '保護裝備名稱', '行動電話或電腦或其他相類功能裝置名稱', '肇事逃逸類別名稱-是否肇逃',

    # 設施
    'youbike_100m_count', 'mrt_100m_count', 'parkinglot_100m_count',

    # A1 or A2
    # 'source',
    ]

In [ ]:
def extract_features(
        grid, combined_data, select_group, rows
        ):

    indices = grid['accident_indices'].iloc[rows] # return list of original data index
    sample = combined_data.iloc[indices]
    sample = sample[select_group]

    cat_cols = sample.select_dtypes(include='object').columns
    num_cols = sample.select_dtypes(include='number').columns

    cat_features = []
    num_features = []
    # for categorical features
    if len(cat_cols) > 0:
        for col in cat_cols:
            vc = sample[col].value_counts(normalize=True)
            vc.index = [f"{col}_{v}" for v in vc.index]
            cat_features.append(vc)
        cat_features = pd.concat(cat_features)
    else:
        cat_features = pd.Series(dtype='float64')
    # for numerical features
    if len(num_cols) > 0:
        num_features = sample[num_cols].mean()
        num_features.index = [f"{col}_mean" for col in num_features.index]
    else:
        num_features = pd.Series(dtype='float64')

    all_features = pd.concat([cat_features, num_features])
    all_features_df = all_features.to_frame().T

    return all_features_df

## Features concat

In [ ]:
all_features_list = []

grid_filter = grid[grid['accident_indices'].str.len() > 0]
for rows in range(grid_filter.shape[0]):
    features = extract_features(grid_filter, combined_data, select_group, rows)
    all_features_list.append(features)

all_features_df = pd.concat(all_features_list, ignore_index=True)
all_features_df.fillna(0, inplace=True)

all_features_df[['mrt_100m_count_mean', 'youbike_100m_count_mean', 'parkinglot_100m_count_mean']] =\
      all_features_df[['mrt_100m_count_mean', 'youbike_100m_count_mean', 'parkinglot_100m_count_mean']].\
        apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [ ]:
# with county town
# 原始資料index並非從1開始所以需reset
new_grid = pd.concat([grid_filter.reset_index(drop=True)[['hotspot', 'COUNTYNAME']], all_features_df], axis=1)
county_dummies = pd.get_dummies(new_grid['COUNTYNAME'], prefix='county')
new_grid_encoded = pd.concat([new_grid.drop(['COUNTYNAME'], axis=1), county_dummies], axis=1)

# without county town
# new_grid = pd.concat([grid_filter.reset_index(drop=True)[['hotspot']], all_features_df], axis=1)
# new_grid_encoded = new_grid

# binary hotspot
new_grid_encoded['hotspot'] = new_grid_encoded['hotspot'].apply(lambda x: 'Hotspot' if 'Hotspot' in str(x) else 'Not Hotspot')

le = LabelEncoder()
y = le.fit_transform(new_grid_encoded['hotspot'])
X = new_grid_encoded.drop(columns=['hotspot'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)
y_train = pd.Series(y_train, index=X_train.index)
y_test  = pd.Series(y_test,  index=X_test.index)

# with undersampling
cls_counts = y_test.value_counts()
min_count = cls_counts.min()
rus_test = RandomUnderSampler(
    sampling_strategy={int(c): int(min_count) for c in cls_counts.index},
    random_state=42
)
X_resampled_test, y_resampled_test = rus_test.fit_resample(X_test, y_test)

# without undersampling
# X_resampled_test = X_test.copy()
# y_resampled_test = y_test.copy()

print("before US")
print(pd.Series(y_test).map(dict(enumerate(le.classes_))).value_counts())
print("after US")
print(pd.Series(y_resampled_test).map(dict(enumerate(le.classes_))).value_counts())

In [ ]:
lr = LogisticRegression(
        class_weight='balanced', max_iter=1000, 
        random_state=42, 
        multi_class='multinomial'
    )
rf = RandomForestClassifier(
        n_estimators=300, max_depth=None, min_samples_leaf=1,
        class_weight='balanced', n_jobs=-1, random_state=42,
    )

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, clf in [('Logistic', lr), ('RandomForest', rf)]:
    scores = cross_val_score(clf, X_train, y_train, cv=cv, n_jobs=-1,
                            scoring='roc_auc_ovr_weighted',
                            # scoring='roc_auc'
                             )
    print(f'{name} CV ROC AUC: {scores.mean():.3f} ± {scores.std():.3f}')

lr.fit(X_train, y_train)
rf.fit(X_train, y_train)

proba_test_lr = lr.predict_proba(X_resampled_test)
proba_test_rf = rf.predict_proba(X_resampled_test)
y_pred_lr = np.argmax(proba_test_lr, axis=1)
y_pred_rf = np.argmax(proba_test_rf, axis=1)

In [ ]:
from sklearn.preprocessing import label_binarize

y_pred = y_pred_rf
proba_test = proba_test_rf

print("Confusion Matrix")
print(confusion_matrix(y_resampled_test, y_pred, labels=range(len(le.classes_))))

print("Classification Report")
print(classification_report(
    y_resampled_test, y_pred, target_names=le.classes_, digits=3
))


if proba_test.shape[1] == 2:
    # 二元分類
    roc_auc = roc_auc_score(y_resampled_test, proba_test[:, 1])
    print(f'ROC AUC: {roc_auc:.3f}')
    y_test_bin = label_binarize(y_resampled_test, classes=range(len(le.classes_)))
    pr_auc_macro  = average_precision_score(y_test_bin, proba_test[:, 1], average='macro')
    pr_auc_weight = average_precision_score(y_test_bin, proba_test[:, 1], average='weighted')
    print(f'PR  AUC macro: {pr_auc_macro:.3f}')
    print(f'PR  AUC wighted: {pr_auc_weight:.3f}')
else:
    # 多類分類
    roc_auc = roc_auc_score(y_resampled_test, proba_test, average='weighted', multi_class='ovr')
    print(f'ROC AUC: {roc_auc:.3f}')
    # 多類PR AUC需要 binarize 後用 one-vs-rest，再做 macro/weighted 平均
    y_test_bin = label_binarize(y_resampled_test, classes=range(len(le.classes_)))  # shape [n, n_classes]
    pr_auc_macro  = average_precision_score(y_test_bin, proba_test, average='macro')
    pr_auc_weight = average_precision_score(y_test_bin, proba_test, average='weighted')
    print(f'PR  AUC macro: {pr_auc_macro:.3f}')
    print(f'PR  AUC wighted: {pr_auc_weight:.3f}')

設施平均：該地區的事故點附近平均會有幾個設施

# RandomForest Feature Importance

In [ ]:
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
feature_names = X_train.columns

for i in indices:
    print(f"{feature_names[i]}: {importances[i]:.4f}")

In [ ]:
feature_names = X_train.columns
importances = rf.feature_importances_

fi_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
fi_df['main_feature'] = fi_df['feature'].str.split('_').str[0]
grouped = fi_df.groupby('main_feature')['importance'].sum().sort_values(ascending=False)

print(grouped)

# LinearRegression coefficient

In [ ]:
coefs = lr.coef_[0]  # 若是二元分類，取第0個類別
feature_names = X_train.columns
indices = np.argsort(np.abs(coefs))[::-1]  # 依照絕對值排序

for i in indices:
    print(f"{feature_names[i]}: {coefs[i]:.4f}")